In [38]:
import pandas as pd
import numpy as np

Tabella dei fondi e dei tagli che devi applicare per ottenere i numeri finali.

Dataset         Events/day      Selection_eff   Muon_cut    Fiducial_volume     Combined_efficiency\
Reactor         57.4            98.4%           91.6%       91.2%               82%\
Geo-U           1.155           98.4%           91.6%       91.2%               82%\
Geo-Th          0.345           98.4%           91.6%       91.2%               82%\
Li9             0.81            98.4%           -           91.2%               90%\
He8             0.09            98.4%           -           91.2%               90%\
World_reactors  1.22            98.4%           91.6%       91.2%               82%\
Atmospheric_nu  0.2             98.4%           91.6%       91.2%               82% (?)\
Fast_neutron    0.12            98.4%           91.6%       91.2%               82%\
CO_alpha_n      0.06            98.4%           91.6%       91.2%               82% (?)\
Accidentals     8.8e6           1.41e-6         91.6%       -                   1.29e-6\

Puoi farlo per ogni categoria, poi sommi tutti gli eventi finali e fai:\
efficiency = selected_IBDs/overall_IBDs\
purity = selected_IBDs/selected_events\

In [39]:
import pandas as pd

data = {
    'Reactor': {
        'evday': 57.4,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Geo-U': {
        'evday': 1.155,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Geo-Th': {
        'evday': 0.345,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Li9': {
        'evday': 0.81,
        'selection_eff': 0.984,
        'muon_cut': 1,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.9
    },
    'He8': {
        'evday': 0.09,
        'selection_eff': 0.984,
        'muon_cut': 1,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.9
    },
    'World_reactors': {
        'evday': 1.22,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Atmospheric_nu': {
        'evday': 0.2,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Fast_neutron': {
        'evday': 0.12,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'CO_alpha_n': {
        'evday': 0.06,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Accidentals': {
        'evday': 134124, #corrisponde alle coincidenze accidentali attese con 39.4 Hz in una finestra di 5 tau
        'selection_eff': 1.41e-6,
        'muon_cut': 0.916,
        'fiducial_volume': 1,
        'combined_efficiency': 1.29e-6
    }
}

df = pd.DataFrame.from_dict(data, orient='index')

#Ricorda, hai messo 1 ai fiducial volume che mancano


In [40]:
eff_test_manual_cut_IBD =  0.9770288355028044
eff_test_xgboost_IBD = 0.9999850163321979
eff_test_pytorch_IBD =   0.9998498167764673

In [41]:
eff_test_manual_cut_BKG = 0.9999750650801408
eff_test_xgboost_BKG = 0.9999799814828717
eff_test_pytorch_BKG =   0.9997703181094184

In [42]:
df

,evday,selection_eff,muon_cut,fiducial_volume,combined_efficiency
Reactor,57.400,0.984000,0.916,0.912,0.820000
Geo-U,1.155,0.984000,0.916,0.912,0.820000
Geo-Th,0.345,0.984000,0.916,0.912,0.820000
Li9,0.810,0.984000,1.000,0.912,0.900000
He8,0.090,0.984000,1.000,0.912,0.900000
World_reactors,1.220,0.984000,0.916,0.912,0.820000
Atmospheric_nu,0.200,0.984000,0.916,0.912,0.820000
Fast_neutron,0.120,0.984000,0.916,0.912,0.820000
CO_alpha_n,0.060,0.984000,0.916,0.912,0.820000
Accidentals,134124.000,0.000001,0.916,1.000,0.000001


IBD

In [43]:
n_IBD_manualcut = np.array([])
n_IBD_xgboost = np.array([])
n_IBD_pytorch = np.array([])
all_IBD_reac = df["evday"]['Reactor']


print("------Manual Cut------", '\t', "-----XGBoost------",'\t', "-------PyTorch----")
for name in df.index.tolist():
    if name!="Accidentals":
        n_IBD_manualcut = np.append(n_IBD_manualcut,df["evday"][name]*eff_test_manual_cut_IBD*df["muon_cut"][name])
        n_IBD_xgboost = np.append(n_IBD_xgboost, df["evday"][name]*eff_test_xgboost_IBD*df["muon_cut"][name])
        n_IBD_pytorch = np.append(n_IBD_pytorch,df["evday"][name]*eff_test_pytorch_IBD*df["muon_cut"][name])
        print(name, '\t', df["evday"][name]*eff_test_manual_cut_IBD*df["muon_cut"][name], '\t',df["evday"][name]*eff_test_xgboost_IBD*df["muon_cut"][name],"\t", df["evday"][name]*eff_test_pytorch_IBD*df["muon_cut"][name])
    
sel_IBD_manualcut = n_IBD_manualcut.sum()
sel_IBD_pytorch = n_IBD_pytorch.sum()
sel_IBD_xgboost = n_IBD_xgboost.sum()


print("---------------", '\t', "---------------")
print("Attesi: ", sel_IBD_manualcut, '\t', sel_IBD_xgboost, '\t', sel_IBD_pytorch)

------Manual Cut------ 	 -----XGBoost------ 	 -------PyTorch----
Reactor 	 51.37061292460065 	 52.57761218272084 	 52.57050360639981
Geo-U 	 1.033676967385257 	 1.0579641475791388 	 1.0578211091531668
Geo-Th 	 0.30876065259559626 	 0.3160152648613012 	 0.3159725390976992
Li9 	 0.7913933567572716 	 0.8099878632290803 	 0.8098783515889386
He8 	 0.08793259519525239 	 0.08999865146989781 	 0.08998648350988206
World_reactors 	 1.091849264251094 	 1.1175032554515578 	 1.1173521672440376
Atmospheric_nu 	 0.1789916826641138 	 0.1831972549920587 	 0.18317248643344886
Fast_neutron 	 0.10739500959846826 	 0.1099183529952352 	 0.10990349186006929
CO_alpha_n 	 0.05369750479923413 	 0.0549591764976176 	 0.05495174593003464
--------------- 	 ---------------
Attesi:  55.02430995784693 	 56.31715614979673 	 56.30954198121709


Questi appena calcolati dovrebbero essere gli eventi che mi ASPETTO DI SELEZIONARE per un giorno di presa dati.\
Gli algoritmi sviluppati restituiscono una accuracy (Che dovrebbe essere l'efficency come la intendiamo noi, ovvero TP/(TP+FN)). Questa accuracy dovrebbe tener conto dei fattori di fiducial volume e anche dei criteri di selection per come è stato valutato l'apprendimento. Dunque infine quello che abbiamo fatto è ottenere un valore migliorato di 'fiducial_volume' * 'selection_effects'. \
Il muon cut non l'abbiamo proprio considerato, dunque lo teniamo per calcolare gli IBD attesi.

Mi mancano i selected events. Ho una efficency per gli eventi di BKG, devo usare quella?

BKG

In [44]:
all_events = pd.read_csv("all_features.csv")

In [45]:
n_ev = all_events["Label"][(all_events["Label"] == 0) & (all_events["Source"] == 0)].shape[0]
n_ev

993457

In [46]:
print("------Manual Cut------", '\t', "-----XGBoost------",'\t', "-------PyTorch----")

FN_manual_cut = n_ev*(1-eff_test_manual_cut_BKG)*df["muon_cut"]["Accidentals"]
FN_xgboost =  n_ev*(1-eff_test_xgboost_BKG)*df["muon_cut"]["Accidentals"]
FN_pytorch = n_ev*(1-eff_test_pytorch_BKG)*df["muon_cut"]["Accidentals"]


print("FP: ", FN_manual_cut, '\t', FN_xgboost, '\t', FN_pytorch)

------Manual Cut------ 	 -----XGBoost------ 	 -------PyTorch----
FP:  22.69094194158104 	 18.216982949183855 	 209.01203908590747


In [47]:
TP_reac_manualcut = df["evday"]['Reactor']*eff_test_manual_cut_IBD*df["muon_cut"]['Reactor']
TP_reac_xgboost =  df["evday"]['Reactor']*eff_test_xgboost_IBD*df["muon_cut"]['Reactor']
TP_reac_pytorch = df["evday"]['Reactor']*eff_test_pytorch_IBD*df["muon_cut"]['Reactor']

In [48]:
final_efficency_manualcut = TP_reac_manualcut/all_IBD_reac
final_efficency_xgboost = TP_reac_xgboost/all_IBD_reac
final_efficency_pytorch = TP_reac_pytorch/all_IBD_reac

# final_purity_pytorch = sel_IBD_pytorch/sel_events

print(final_efficency_manualcut)
print(final_efficency_xgboost)
print(final_efficency_pytorch)

0.8949584133205688
0.9159862749602934
0.9158624321672442


In [49]:
final_purity_manualcut = TP_reac_manualcut/(sel_IBD_manualcut + FN_manual_cut)
final_purity_xgboost = TP_reac_xgboost/(sel_IBD_xgboost + FN_xgboost)
final_purity_pytorch = TP_reac_pytorch/(sel_IBD_pytorch + FN_pytorch)

# final_purity_pytorch = sel_IBD_pytorch/sel_events

print(final_purity_manualcut)
print(final_purity_xgboost)
print(final_purity_pytorch)

0.6610106982742568
0.7054165087074837
0.1981388147732311
